### Imports

In [ ]:
import math
import scipy
from scipy.sparse import csgraph
from sklearn.cluster import DBSCAN
from scipy import linalg as LA
import numpy as np
import pandas as pd
from sklearn.neighbors import kneighbors_graph

### Getting affinity matrix of coordinates

In [ ]:
def getAffinityMatrix(coordinates, n_neighbors, mode='fill_symmetric', LD=None):
  A = kneighbors_graph(coordinates, n_neighbors=min(n_neighbors,len(coordinates)-1), mode='connectivity', include_self=False).toarray()
  if mode=='fill_symmetric':
    for i in range(len(A)):
      for j in range(len(A[0])):
        if A[i][j] == 1:
          A[j][i] = 1
  elif mode=='mutal_symmetric':
    for i in range(len(A)):
      for j in range(len(A[0])):
        if A[i][j] != A[j][i]:
          A[j][i] = 0
  elif mode=='ld_constraint':
    # if LD is None:
    #   LD = get_local_intrinsic_dimension(coordinates,n_neighbors)
    for i in range(len(A)):
      for j in range(len(A[0])):
        if LD[i] != LD[j]:
          A[i][j] = 0
          A[j][i] = 0
  return A



### Getting knapsacks cost function from affinity matrix

In [ ]:
def getManifoldKSimilarity(list_manifold, list_k):
  m = len(list_manifold)
  num_k = len(list_k)
  TopK = max(list_k)+2
  M = []
  for i in range(m):
    print(i)
    A = list_manifold[i]
    L = csgraph.laplacian(A, normed=True)
    n_components = A.shape[0]-1
    eigenvalues = scipy.sparse.linalg.eigsh(L,k=min(TopK,n_components), return_eigenvectors=False, which='SM',
                                            tol=1e-3, maxiter=500)
    eigenvalues = eigenvalues[::-1]
    # eigenvalues = sorted(eigenvalues)

    # if plot:
    #     plt.title('Largest eigen values of input matrix')
    #     plt.scatter(np.arange(len(eigenvalues))[:10], eigenvalues[:10])
    #     plt.grid()

    max_gap = 0
    gap_pre_index = 0
    k_gap = [0 for i in range(len(list_k))]
    k_gap_pre_index = [0 for i in range(len(list_k))]
    for i in range(1, TopK):
        gap = np.abs(eigenvalues[i] / eigenvalues[i - 1])
        if i in list_k:
          k_gap[list_k.index(i)] = gap
          k_gap_pre_index[list_k.index(i)] = i
        if gap > max_gap:
            max_gap = gap
            gap_pre_index = i
    k_gap_similarity = [k_gap[i]/max_gap * 1/(np.abs(k_gap_pre_index[i]-gap_pre_index)+1) for i in range(len(k_gap))]
    print(k_gap_similarity)
    M.append(k_gap_similarity)
  return M

### Knapsack

In [ ]:
# cost is m * len(k_list) showing cost for each k_i option
# m_size is the number of manifolds / subsets
# k_sum is the required sum of chosen k_is
# k_list is the list of possible values for k_is
def finding_optimal_ks(cost, m_size, k_sum, k_list):
    dp = []
    dp_base = [0]
    par = []
    par_base = [0]
    k_size = len(k_list)
    for i in range(1, k_sum + 1):
        dp_base.append(math.inf)
        par_base.append(-1)
    dp.append(dp_base)
    par.append(par_base)
    for i in range(1, m_size + 1):
        tmp_dp_list = [math.inf] * (k_sum + 1)
        tmp_par_list = [-1] * (k_sum + 1)
        for j in range(k_sum + 1):
            for z in range(k_size):
                if j >= k_list[z]:
                    print(k_sum, ' ', len(dp[i-1]))
                    if tmp_dp_list[j] > dp[i - 1][j - k_list[z]] + cost[i-1][z]:
                        if par[i - 1][j - k_list[z]] != - 1:
                            tmp_dp_list[j] = dp[i - 1][j - k_list[z]] + cost[i-1][z]
                            tmp_par_list[j] = z
        dp.append(tmp_dp_list)
        par.append(tmp_par_list)
    ind = m_size
    k_ind = k_sum
    res = []
    while ind != 0:
        res.append(par[ind][k_ind])
        k_ind -= k_list[par[ind][k_ind]]
        ind -= 1
    res.reverse()
    return res, dp[m_size][k_sum]